## Рекомендательная система

### Этап 1. Подготовка данных

In [66]:
# эти библиотеки нам уже знакомы
import pandas as pd
import numpy as np

# модуль sparse библиотеки scipy понадобится
# для работы с разреженными матрицами (об этом ниже)
from scipy.sparse import csr_matrix

# из sklearn мы импортируем алгоритм k-ближайших соседей
from sklearn.neighbors import NearestNeighbors

Для начала прочитаем внешние файлы

In [67]:
# прочитаем внешние файлы (перед этим их необходимо импортировать) и преобразуем в датафрейм
movies = pd.read_csv('cellphones_data.csv')
ratings = pd.read_csv('cellphones_ratings.csv')

In [68]:
# посмотрим на содержимое файла movies.csv
# дополнительно удалим столбец genres, он нам не нужен
# (параметр axis = 1 говорит, что мы работаем со столбцами, inplace = True, что изменения нужно сохранить)
movies.drop(['RAM', 'performance', 'main camera',
            'selfie camera', 'battery size', 'screen size', 'weight', 'price', 'release date'], axis = 1, inplace = True)
movies.head(3)

,cellphone_id,brand,model,operating system,internal memory
0,0,Apple,iPhone SE (2022),iOS,128
1,1,Apple,iPhone 13 Mini,iOS,128
2,2,Apple,iPhone 13,iOS,128


In [69]:
# и ratings.csv (здесь также удаляем ненужный столбец timestamp)
# ratings.drop(['timestamp'], axis = 1, inplace = True)
ratings.head(3)

,user_id,cellphone_id,rating
0,0,30,1
1,0,5,3
2,0,10,9


In [70]:
# для этого воспользуемся функцией pivot и создадим сводную таблицу (pivot table)
# по горизонтали будут фильмы, по вертикали - пользователи, значения - оценки
user_item_matrix = ratings.pivot(index = 'cellphone_id', columns = 'user_id', values= 'rating')
user_item_matrix.head()

user_id,0,1,6,8,10,12,16,24,25,26,...,245,246,251,252,253,254,255,256,257,258
cellphone_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,8.0,NaN,NaN,8.0,NaN,NaN,NaN,7.0,NaN
1,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,10.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,6.0,NaN,7.0,NaN,NaN,NaN,NaN,7.0,6.0,NaN
3,10.0,10.0,NaN,NaN,9.0,NaN,NaN,10.0,NaN,10.0,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
4,NaN,7.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,9.0,...,8.0,NaN,8.0,6.0,8.0,NaN,10.0,5.0,8.0,NaN


In [71]:
# пропуски NaN нужно преобразовать в нули
# параметр inplace = True опять же поможет сохранить результат
user_item_matrix.fillna(0, inplace = True)
user_item_matrix.head()

user_id,0,1,6,8,10,12,16,24,25,26,...,245,246,251,252,253,254,255,256,257,258
cellphone_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,7.0,0.0
1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,10.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,6.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,6.0,0.0
3,10.0,10.0,0.0,0.0,9.0,0.0,0.0,10.0,0.0,10.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,0.0,7.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,9.0,...,8.0,0.0,8.0,6.0,8.0,0.0,10.0,5.0,8.0,0.0


In [72]:
# посмотрим на размерность матрицы "пользователи х фильмы"
user_item_matrix.shape

(33, 99)

Теперь давайте уберем неактивных пользователей и фильмы с небольшим количеством оценок

In [59]:
# вначале сгруппируем (объединим) пользователей, возьмем только столбец rating
# и посчитаем, сколько было оценок у каждого пользователя
users_votes = ratings.groupby('user_id')['rating'].agg('count')

# сделаем то же самое, только для фильма
movies_votes = ratings.groupby('cellphone_id')['rating'].agg('count')

In [60]:
# теперь создадим фильтр (mask)
user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 1].index

In [61]:
# применим фильтры и отберем фильмы с достаточным количеством оценок
user_item_matrix = user_item_matrix.loc[movie_mask,:]

# а также активных пользователей
user_item_matrix = user_item_matrix.loc[:,user_mask]

In [73]:
# посмотрим сколько пользователей и фильмов осталось
user_item_matrix.shape

(33, 99)

Мы почти завершили первый этап. В частности, осталось преобразовать нашу разреженную матрицу (sparce matrix) в сжатое хранение строкой (сompressed sparse row) с помощью функции csr_matrix библиотеки Scipy.

In [74]:
# преобразуем разреженную матрицу в формат csr
# метод values передаст функции csr_matrix только значения датафрейма
csr_data = csr_matrix(user_item_matrix.values)

# посмотрим на первые записи
# сопоставьте эти значения с исходной таблицей выше
print(csr_data[:2,:5])

  (0, 3)	5.0
  (1, 2)	2.0


In [75]:
# остается только сбросить индекс с помощью reset_index()
# это необходимо для удобства поиска фильма по индексу
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,cellphone_id,0,1,6,8,10,12,16,24,25,...,245,246,251,252,253,254,255,256,257,258
0,0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,...,0.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,7.0,0.0
1,1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,10.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,6.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,6.0,0.0
3,3,10.0,10.0,0.0,0.0,9.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,7.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,8.0,0.0,8.0,6.0,8.0,0.0,10.0,5.0,8.0,0.0


### Этап 2. Обучение модели

In [76]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)

# обучим модель
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

### Этап 3. Составление рекомендаций

Зададим изначальные параметры поиска

In [77]:
# ждя начала определимся, сколько рекомендаций мы хотим получить
recommendations = 10

# и на основе какого фильма
search_word = 'OnePlus'

Найдем индекс фильма в матрице предпочтений

In [78]:
# для начала найдем фильм в заголовках датафрейма movies
movie_search = movies[movies['brand'].str.contains(search_word)]
movie_search

,cellphone_id,brand,model,operating system,internal memory
18,18,OnePlus,Nord N20,Android,128
19,19,OnePlus,Nord 2T,Android,128
20,20,OnePlus,10 Pro,Android,128
21,21,OnePlus,10T,Android,128


In [ ]:
# вариантов может быть несколько, для простоты всегда будем брать первый вариант
# через iloc[0] мы берем первую строку столбца ['movieId']
movie_id = movie_search.iloc[0]['movieId']

# далее по индексу фильма в датасете movies найдем соответствующий индекс
# в матрице предпочтений
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]
movie_id

Находим схожие фильмы

In [ ]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recommendations + 1)

In [ ]:
# индексы рекомендованных фильмов
indices

In [ ]:
# расстояния до них
distances

In [ ]:
# уберем лишние измерения через squeeze() и преобразуем массивы в списки с помощью tolist()
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

# далее с помощью функций zip и list преобразуем наши списки
indices_distances = list(zip(indices_list, distances_list))

# в набор кортежей (tuple)
print(type(indices_distances[0]))

# и посмотрим на первые три пары/кортежа
print(indices_distances[:3])

In [ ]:
# остается отсортировать список по расстояниям через key = lambda x: x[1] (то есть по второму элементу)
# в возрастающем порядке reverse = False
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)

# и убрать первый элемент с индексом 901 (потому что это и есть "Матрица")
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

Остается найти какие фильмы соответствуют найденным нами индексам

In [ ]:
# создаем пустой список, в который будем помещать название фильма и расстояние до него
recom_list = []

# теперь в цикле будем поочередно проходить по кортежам
for ind_dist in indices_distances_sorted:

    # искать movieId в матрице предпочтений
    matrix_movie_id = user_item_matrix.iloc[ind_dist[0]]['movieId']

    # выяснять индекс этого фильма в датафрейме movies
    id = movies[movies['movieId'] == matrix_movie_id].index

    # брать название фильма и расстояние до него
    title = movies.iloc[id]['title'].values[0]
    dist = ind_dist[1]

    # помещать каждую пару в питоновский словарь
    # который, в свою очередь, станет элементом списка recom_list
    recom_list.append({'Title' : title, 'Distance' : dist})

In [ ]:
# посмотрим на первый элемент
recom_list[0]

In [ ]:
# остается преобразовать наш список в датафрейм
# индекс будем начинать с 1, как и положено рейтингу
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

### Ответы на вопросы

**Вопрос**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Векторы на графике

In [ ]:
# создадим два вектора с координатами [10, 10] и [1, 1]
x = np.array([10, 10])
y = np.array([1, 1])

In [ ]:
# зададим размер фигуры (контейнера, в который помещаются графики)
plt.figure(figsize = (10, 6))

# создадим объект графика внутри этой фигуры
ax = plt.axes()

# зададим диапазон осей x и y
plt.xlim([0, 11])
plt.ylim([0, 11])
plt.grid()

# нашими "подграфиками" будут два вектора в форме стрелок
ax.arrow(0, 0, x[0], x[1], width = 0.03, head_width = 0.2, head_length = 0.2, fc = 'g', ec = 'g')
ax.arrow(0, 0, y[0], y[1], width = 0.03, head_width = 0.2, head_length = 0.2, fc = 'b', ec = 'b')

plt.show()

Расчет косинусного сходства

In [ ]:
# напишем функцию для расчета косинусного сходства
def similar(x, y):

    # рассчитаем длины векторов
    xLen = np.linalg.norm(x)
    yLen = np.linalg.norm(y)

    # подставим их в формулу косинусного сходства
    result = np.dot(x, y)/(xLen * yLen)

    # выведем результат
    return result

In [ ]:
# ожидаемо косунус угла будет равен единице
round(similar(x, y), 3)

Item-based система

In [ ]:
# создадим массив Numpy с оценками
films = np.array(
    [
     [1, 3, 2, 6, 2, 1, 0],
     [0, 2, 0, 3, 0, 6, 2],
     [1, 1, 1, 1, 1, 1, 1],
     [2, 4, 1, 3, 9, 2, 1],
     [10, 10, 10, 10, 10, 10, 10]
     ]
)

# строки это фильмы, столбцы - пользователи
films

In [ ]:
# предположим, вышел новый фильм, и все пользователи поставили ему рейтинг 10
new_film = np.array([10, 10, 10, 10, 10, 10, 10])

In [ ]:
# в цикле for поочередно рассчитаем косинусное сходство каждого из имеющхся фильмов с новым фильмом
for i, film in enumerate(films, 1):
  print(f'Фильм {i} с оценками {film} имеет сходство с новым фильмом {np.round(similar(film, new_film), 3)}')